In [ ]:
!pip install optapy

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time
from optapy import problem_fact, planning_id
from typing import List, Dict, Tuple

In [2]:
area_path = '../dataset/area_work/'
path_to_facility= '../dataset/facility_timing/'

In [3]:
# Read visibility to facilites data
dfs = []
for path, subdirs, files in os.walk(path_to_facility):
    for name in files:
        if '.csv' in name:
            facility_csv_path = os.path.join(path, name)
            df = pd.read_csv(facility_csv_path, index_col=0)
            df['Start Time (UTCG)'] = pd.to_datetime(df['Start Time (UTCG)'], format='%d %b %Y %H:%M:%S.%f')
            df['Stop Time (UTCG)'] = pd.to_datetime(df['Stop Time (UTCG)'], format='%d %b %Y %H:%M:%S.%f')
            naming = name.split('-To-')
            df['facility'] = naming[0]
            sat_name = naming[1].split('.')[0]
            orbit_num = int(sat_name.split('_11')[1][:-2])
            if orbit_num > 5:
                sat_name = 'ZorkySat_' + sat_name.split('_')[1]
            df['sat_name'] = sat_name
            dfs.append(df)

facilities_df = pd.concat(dfs)
facilities_df = facilities_df.sort_values(by=['Start Time (UTCG)'])

In [4]:
# Read area data
dfs = []
for path, subdirs, files in os.walk(area_path):
    for name in files:
        if '_' in name:
            area_csv_path = os.path.join(path, name)
            df = pd.read_csv(area_csv_path, index_col=0)
            df['Start Time (UTCG)'] = pd.to_datetime(df['Start Time (UTCG)'], format='%d %b %Y %H:%M:%S.%f')
            df['Stop Time (UTCG)'] = pd.to_datetime(df['Stop Time (UTCG)'], format='%d %b %Y %H:%M:%S.%f')
            df['sat_name'] = name.split('.')[0]
            dfs.append(df)

area_df = pd.concat(dfs)
area_df = area_df.sort_values(by=['Start Time (UTCG)'])

In [5]:
lower_bound = area_df['Start Time (UTCG)'].min()
upper_bound = area_df[area_df['Start Time (UTCG)'] > datetime(2027, 6, 3)]['Start Time (UTCG)'].min()

In [6]:
area_df = area_df[(area_df['Start Time (UTCG)'] >= lower_bound) & (area_df['Start Time (UTCG)'] < upper_bound)]
facilities_df = facilities_df[(facilities_df['Start Time (UTCG)'] > lower_bound) & (facilities_df['Start Time (UTCG)'] < upper_bound)]

In [7]:
#calculate data filmed by sat (4Gbit/sec) in Tbytes
area_df['Data'] = area_df['Duration (sec)'] * 4 / 8192

In [8]:
sat_names = area_df['sat_name'].unique()
dfs = []
i = 0
for sat_name in sat_names:
    i += 1
    print(f'{i}/{len(sat_names)} - {sat_name}')
    area_df_sat = area_df[area_df['sat_name'] == sat_name]
    facilities_df_sat = facilities_df[facilities_df['sat_name'] == sat_name]
    filtered_facilities_df = facilities_df_sat.copy()
    for _, area_row in area_df_sat.iterrows():
        cut_start = area_row['Start Time (UTCG)']
        cut_stop = area_row['Stop Time (UTCG)']
        new_df = pd.DataFrame(columns=filtered_facilities_df.columns)
        for index, facility_row in filtered_facilities_df.iterrows():
            start_time_facility = facility_row['Start Time (UTCG)']
            stop_time_facility = facility_row['Stop Time (UTCG)']
            if max(cut_start, start_time_facility) < min(cut_stop, stop_time_facility):
                alt_row = facility_row.copy()
                if start_time_facility < cut_start:
                    facility_row['Stop Time (UTCG)'] = cut_start
                    new_df = pd.concat([new_df, facility_row.to_frame(1).T], ignore_index=True)
                if stop_time_facility > cut_stop:
                    alt_row['Start Time (UTCG)'] = cut_stop
                    new_df = pd.concat([new_df, alt_row.to_frame(1).T], ignore_index=True)
            else:
                new_df = pd.concat([new_df, facility_row.to_frame(1).T], ignore_index=True)
        filtered_facilities_df = new_df.copy()
    dfs.append(filtered_facilities_df)

facilities_df_no_area = pd.concat(dfs)

1/200 - ZorkySat_111607
2/200 - ZorkySat_111806
3/200 - ZorkySat_111008
4/200 - KinoSat_110202
5/200 - ZorkySat_111109
6/200 - ZorkySat_111706
7/200 - ZorkySat_112005
8/200 - ZorkySat_111507
9/200 - ZorkySat_111308
10/200 - KinoSat_110301
11/200 - KinoSat_110102
12/200 - KinoSat_110510
13/200 - ZorkySat_111905
14/200 - ZorkySat_111009
15/200 - ZorkySat_110709
16/200 - ZorkySat_110908
17/200 - ZorkySat_111208
18/200 - KinoSat_110410
19/200 - ZorkySat_111407
20/200 - ZorkySat_110609
21/200 - KinoSat_110201
22/200 - ZorkySat_111606
23/200 - ZorkySat_111805
24/200 - ZorkySat_110909
25/200 - KinoSat_110310
26/200 - ZorkySat_110808
27/200 - ZorkySat_112004
28/200 - ZorkySat_111108
29/200 - ZorkySat_111307
30/200 - KinoSat_110101
31/200 - ZorkySat_111007
32/200 - ZorkySat_111506
33/200 - ZorkySat_111705
34/200 - ZorkySat_110708
35/200 - KinoSat_110509
36/200 - ZorkySat_111904
37/200 - KinoSat_110409
38/200 - ZorkySat_110907
39/200 - ZorkySat_111207
40/200 - KinoSat_110210
41/200 - ZorkySat_11

In [9]:
facilities_df_no_area.reset_index(inplace=True, drop=True)
test_df = facilities_df_no_area.copy()
test_df['Duration (sec)'] = (test_df['Stop Time (UTCG)'] - test_df['Start Time (UTCG)']).dt.total_seconds()
final_no_aree_facilities_df = test_df.copy()

In [10]:
# delete time intervalsthat are too short
seconds_to_split = 30
filtered_by_grain = final_no_aree_facilities_df[final_no_aree_facilities_df['Duration (sec)'] >= seconds_to_split]

In [11]:
@problem_fact
class TimeGrain:
    def __init__(self, id: int, start_time: np.datetime64, end_time: np.datetime64):
        self.id = id
        self.start_time = start_time
        self.end_time = end_time

    @planning_id
    def get_id(self):
        return self.id

    @property
    def duration(self):
        return (self.end_time - self.start_time) / np.timedelta64(1, 's')

    def __str__(self):
        return f'TimeGrain(id={self.id}, start_time={self.start_time}, end_time={self.end_time})'

@problem_fact
class Satellite:
    def __init__(self, id: int, name: str, capacity: float, downlink_speed: float, area_pass_to_data_filmed: List[Tuple[np.datetime64, float]]):
        self.id = id
        self.name = name
        self.capacity = capacity
        self.downlink_speed = downlink_speed
        self.area_pass_to_data_filmed = area_pass_to_data_filmed
        self.storage_history_time = {}
        self.storage_data_history = {}

    @planning_id
    def get_id(self):
        return self.id

    def get_count_overflow(self):
        count_overflow = 0
        for key, value in self.storage_data_history.items():
            if value > self.capacity:
                count_overflow += 1
        return count_overflow

    def get_count_over_downlink(self):
        count_over_downlink = 0
        for key, value in self.storage_data_history.items():
            if value < 0:
                count_over_downlink += 1
        return count_over_downlink

    # return last data by time stamp when sat was filming
    def get_data_filmed_before_time(self, time_stamp: np.datetime64) -> float:
        """Returns the last data filmed by sat before time_stamp"""
        sat_data = self.area_pass_to_data_filmed
        for i in range(len(sat_data)):
            if sat_data[i][0] > time_stamp:
                if i-1 >= 0:
                    return sat_data[i-1][1]
                else:
                    return 0
        return sat_data[-1][1]

    def flatten(self, l):
        return [item for sublist in l for item in sublist]

    def create_storage_data_history(self):
        all_time_grains = self.storage_history_time.values()
        all_time_grains = self.flatten(all_time_grains)
        all_time_grains.sort(key=lambda x: x.start_time)
        storage_data_history = {}
        data_filmed = 0 # in Tbytes
        for time_grain in all_time_grains:
            print(time_grain.start_time)
            cur_filmed = self.get_data_filmed_before_time(time_grain.start_time)
            print(cur_filmed)
            if cur_filmed != data_filmed:
                data_filmed = cur_filmed
                if len(storage_data_history) == 0:
                    storage_data_history[time_grain.end_time] = data_filmed - ((time_grain.duration * self.downlink_speed) / 8192)
                else:
                    storage_data_history[time_grain.end_time] = data_filmed - ((time_grain.duration * self.downlink_speed) / 8192) + storage_data_history[list(storage_data_history)[-1]]
            else:
                if len(storage_data_history) == 0:
                    storage_data_history[time_grain.end_time] = 0
                else:
                    storage_data_history[time_grain.end_time] = storage_data_history[list(storage_data_history)[-1]] - ((time_grain.duration * self.downlink_speed) / 8192)
        self.storage_data_history = storage_data_history

    def update_storage_history(self, history: List[TimeGrain], fac_name: str):
        self.storage_history_time[fac_name] = history

    def __str__(self):
        return f'Satellite(id={self.id}, name={self.name})'

@problem_fact
class Facility:
    def __init__(self, id: int, name: str, sattelite_visibility_zones: List[List[Tuple[np.datetime64, np.datetime64]]]):
        self.id = id
        self.name = name
        self.sattelite_visibility_zones = sattelite_visibility_zones

    @planning_id
    def get_id(self):
        return self.id

    def get_sattelite_visibility_zones(self, sat_id):
        return self.sattelite_visibility_zones[sat_id]

    def time_grain_fits_zone(self, sat_id: int, time_grain: TimeGrain):
        sat_visibility = self.sattelite_visibility_zones[sat_id]
        for start, stop in sat_visibility:
            if start <= time_grain.start_time and stop >= time_grain.end_time:
                return True
        return False


    def __str__(self):
        return f'Facility(id={self.id}, name={self.name})'

In [12]:
from optapy import planning_entity, planning_list_variable

def format_list(a_list):
    return ',\n'.join(map(str, a_list))

@planning_entity
class Assignment:
    def __init__(self, id, facility: Facility, satellite: Satellite, time_grains = None):
        self.id = id
        self.facility = facility
        self.satellite = satellite
        if time_grains is None:
            self.time_grains = []
        else:
            self.time_grains = time_grains

    @planning_id
    def get_id(self):
        return self.id

    @planning_list_variable(TimeGrain, ['TimeGrain_range'])
    def get_time_grains(self):
        return self.time_grains

    def set_time_grains(self, time_grains):
        self.time_grains = time_grains
        # self.save_taime_grains()



    def save_taime_grains(self):
        self.satellite.update_storage_history(self.time_grains, self.facility.name)
        self.satellite.create_storage_data_history()

    def total_downloaded_data(self):
        return sum([time_grain.duration * self.satellite.downlink_speed for time_grain in self.time_grains]) / 8192

    def __str__(self):
        return f'Assignment(id={self.id}, facility={self.facility}, satellite={self.satellite}, time_grains={format_list(self.time_grains)})'


In [13]:
from optapy import constraint_provider
from optapy.types import Joiners, HardSoftScore, ConstraintFactory

def count_time_grains_overlaping(time_grains_1, time_grains_2):
    count = 0
    for time_grain_1 in time_grains_1:
        for time_grain_2 in time_grains_2:
            if time_grain_1.start_time < time_grain_2.end_time and time_grain_1.end_time > time_grain_2.start_time:
                count += 1
    return count

def time_grains_fit_vizibility(Assignment):
    for time_grain in Assignment.time_grains:
        if not Assignment.facility.time_grain_fits_zone(Assignment.satellite.get_id(), time_grain):
            return False
    return True


@constraint_provider
def define_constraints(constraint_factory: ConstraintFactory):
    return [
        # Hard constraints
        facility_conflict(constraint_factory),
        satellite_conflict(constraint_factory),
        assignment_time_grains_should_fit_in_visibility_zone(constraint_factory),
        data_should_not_be_downloaded_on_empty_storage(constraint_factory),
        # Soft constraints
        data_storage_optimap_usage(constraint_factory),
        total_data_downloaded_should_be_maximized(constraint_factory),
    ]

def facility_conflict(constraint_factory: ConstraintFactory):
    # A facility can only be assigned to one satellite at a time grain
    return constraint_factory.for_each(Assignment) \
        .join(Assignment, Joiners.equal(lambda a: a.satellite)) \
        .penalize('Facility conflict', HardSoftScore.ONE_HARD, lambda a1, a2: count_time_grains_overlaping(a1.time_grains, a2.time_grains))

def satellite_conflict(constraint_factory: ConstraintFactory):
    # Only one sattelite can be assignet at specific time grain in specific facility
    return constraint_factory.for_each(Assignment) \
        .join(Assignment, Joiners.equal(lambda a: a.facility)) \
        .penalize('Satellite conflict', HardSoftScore.ONE_HARD, lambda a1, a2: count_time_grains_overlaping(a1.time_grains, a2.time_grains))

def assignment_time_grains_should_fit_in_visibility_zone(constraint_factory: ConstraintFactory):
    # Time grains of an assignment should fit in the visibility zone of the facility
    return constraint_factory.for_each(Assignment) \
        .filter(lambda a: not time_grains_fit_vizibility(a)) \
        .penalize('Assignment time grains should fit in visibility zone', HardSoftScore.ONE_HARD)

def data_should_not_be_downloaded_on_empty_storage(constraint_factory: ConstraintFactory):
    # Data should not be downloaded on empty storage
    return constraint_factory.for_each(Assignment) \
        .penalize('Data should not be downloaded on empty storage', HardSoftScore.ONE_HARD, lambda a: a.satellite.get_count_over_downlink())

def data_storage_optimap_usage(constraint_factory: ConstraintFactory):
    # Data storage optimap usage
    return constraint_factory.for_each(Assignment) \
        .penalize('Data storage optimap usage', HardSoftScore.ONE_SOFT, lambda a: a.satellite.get_count_overflow())

def total_data_downloaded_should_be_maximized(constraint_factory: ConstraintFactory):
    # Total data downloaded should be maximized
    return constraint_factory.for_each(Assignment) \
        .reward('Total data downloaded should be maximized', HardSoftScore.ONE_SOFT, lambda a: a.total_downloaded_data())

In [14]:
from optapy import planning_solution, planning_entity_collection_property, \
    problem_fact_collection_property, \
    value_range_provider, planning_score

@planning_solution
class TransmissionSchedule:
    def __init__(self, facilities: List[Facility], satellites: List[Satellite], assignments: List[Assignment], timegrains: List[TimeGrain], score = None):
        self.facilities = facilities
        self.satellites = satellites
        self.assignments = assignments
        self.timegrains = timegrains
        self.score = score

    def init_sat_storage_history(self):
        sat_to_fac_dict = {}
        for assignment in self.assignments:
            if assignment.facility.name not in sat_to_fac_dict:
                sat_to_fac_dict[assignment.facility.name] = []
            if assignment.satellite.name not in sat_to_fac_dict[assignment.facility.name]:
                assignment.satellite.update_storage_history(assignment.time_grains, assignment.facility.name)
                sat_to_fac_dict[assignment.facility.name].append(assignment.satellite.name)
            if len(sat_to_fac_dict[assignment.facility.name]) == 200:
                assignment.satellite.create_storage_data_history()
                sat_to_fac_dict[assignment.facility.name].append('done!')

    @problem_fact_collection_property(Facility)
    def get_facilities(self):
        return self.facilities

    @problem_fact_collection_property(Satellite)
    def get_satellites(self):
        return self.satellites

    @problem_fact_collection_property(TimeGrain)
    @value_range_provider('TimeGrain_range', value_range_type=list)
    def get_time_grains(self):
        return self.timegrains

    @planning_entity_collection_property(Assignment)
    def get_assignments(self):
        return self.assignments

    @planning_score(HardSoftScore)
    def get_score(self):
        return self.score

    def set_score(self, score):
        self.score = score

    def __str__(self):
        return f"TransmissionSchedule(facilities={format_list(self.facilities)}, satellites={format_list(self.satellites)}, assignments={format_list(self.assignments)}, score={str(self.score.toString()) if self.score is not None else 'None'})"

In [15]:
sat_names = area_df['sat_name'].unique()
sat_names = sorted(sat_names)
facilities_name = filtered_by_grain['facility'].unique()
facilities_name = sorted(facilities_name)
# encode sat names with numbers
sat_name_to_num = {name: i for i, name in enumerate(sat_names)}
fac_name_to_num = {name: i for i, name in enumerate(facilities_name)}
data_filmed_by_sat = []
for sat_name in sat_names:
    sat_df = area_df[area_df['sat_name'] == sat_name]
    # append time as numpy datetime64
    data_filmed_by_sat.append(list(zip(sat_df['Stop Time (UTCG)'].to_numpy(), sat_df['Data'])))
visibility_slots = []
for facility_name in facilities_name:
    facility_df = filtered_by_grain[filtered_by_grain['facility'] == facility_name]
    sat_visibility = []
    for sat_name in sat_names:
        sat_df = facility_df[facility_df['sat_name'] == sat_name]
        sat_visibility.append(list(zip(sat_df['Start Time (UTCG)'].to_numpy(), sat_df['Stop Time (UTCG)'].to_numpy())))
    visibility_slots.append(sat_visibility)

In [16]:
def initial_solutioin_generator():
    """Generate initial solution with all facilities assigned to all satellites"""
    facilities: List[Facility] = []
    for i, facility_name in enumerate(facilities_name):
        facilities.append(Facility(i, facility_name, visibility_slots[i]))
    satellites: List[Satellite] = []
    for i, sat_name in enumerate(sat_names):
        if 'Zorky' in sat_name:
            capacity = 0.5
            downlink_speed = 0.25
        else:
            capacity = 1
            downlink_speed = 1
        satellites.append(Satellite(i, sat_name, capacity, downlink_speed, data_filmed_by_sat[i]))
    time_grains = []
    assignments = []
    grain_index = 0
    assignment_index = 0
    for fac_num in range(len(facilities)):
        for sat_num in range(len(satellites)):
            start_grain_index = grain_index
            cur_select_df = filtered_by_grain[(filtered_by_grain['facility'] == facilities[fac_num].name) & (filtered_by_grain['sat_name'] == satellites[sat_num].name)]
            for _, row in cur_select_df.iterrows():
                low_t = row['Start Time (UTCG)']
                up_t = row['Stop Time (UTCG)']
                time_intervals = np.linspace(low_t.value, up_t.value, int((up_t-low_t).total_seconds()) // seconds_to_split)
                time_intervals = np.asarray(time_intervals, dtype='datetime64[ns]')
                time_intervals = np.sort(time_intervals)
                for time_grain_index in range(len(time_intervals)-1):
                    time_interval_start = time_intervals[time_grain_index]
                    time_interval_stop = time_intervals[time_grain_index+1]
                    # round ns to ms
                    time_interval_start = np.datetime64(int(int(int(time_interval_start) / 100000) * 100000), 'ns')
                    time_interval_stop = np.datetime64(int(int(int(time_interval_stop) / 100000) * 100000), 'ns')
                    time_grains.append(TimeGrain(grain_index, time_interval_start, time_interval_stop))
                    grain_index += 1
            assignments.append(Assignment(assignment_index, facilities[fac_num], satellites[sat_num], time_grains[start_grain_index:grain_index]))
            assignment_index += 1
    return TransmissionSchedule(facilities, satellites, assignments, time_grains)

In [17]:
initial_solution = initial_solutioin_generator()

In [ ]:
initial_solution.init_sat_storage_history()

In [18]:
import optapy.config
from optapy.types import Duration

solver_config = optapy.config.solver.SolverConfig() \
    .withEntityClasses(Assignment) \
    .withSolutionClass(TransmissionSchedule) \
    .withConstraintProviderClass(define_constraints) \
    .withTerminationSpentLimit(Duration.ofMinutes(30))

In [29]:
import logging

logging.getLogger('optapy').setLevel(logging.DEBUG)
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='test_log.log', mode='a')
logger.addHandler(fhandler)
logging.warning('This is a warning message')

In [31]:
from optapy import solver_factory_create

solution = solver_factory_create(solver_config) \
    .buildSolver() \
    .solve(initial_solution)

print(solution)

In [24]:
print(initial_solution.assignments[0])

Assignment(id=0, facility=Facility(id=0, name=Anadyr1), satellite=Satellite(id=0, name=KinoSat_110101), time_grains=TimeGrain(id=0, start_time=2027-06-01T09:48:23.799000000, end_time=2027-06-01T09:48:56.147700000),
TimeGrain(id=1, start_time=2027-06-01T09:48:56.147700000, end_time=2027-06-01T09:49:28.496500000),
TimeGrain(id=2, start_time=2027-06-01T09:49:28.496500000, end_time=2027-06-01T09:50:00.845300000),
TimeGrain(id=3, start_time=2027-06-01T09:50:00.845300000, end_time=2027-06-01T09:50:33.194100000),
TimeGrain(id=4, start_time=2027-06-01T09:50:33.194100000, end_time=2027-06-01T09:51:05.542900000),
TimeGrain(id=5, start_time=2027-06-01T09:51:05.542900000, end_time=2027-06-01T09:51:37.891700000),
TimeGrain(id=6, start_time=2027-06-01T09:51:37.891700000, end_time=2027-06-01T09:52:10.240500000),
TimeGrain(id=7, start_time=2027-06-01T09:52:10.240500000, end_time=2027-06-01T09:52:42.589300000),
TimeGrain(id=8, start_time=2027-06-01T09:52:42.589300000, end_time=2027-06-01T09:53:14.93810